In [ ]:
import torch

# Select GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Later: move model and data to this device
model = MyModel().to(device)
inputs = inputs.to(device)
labels = labels.to(device)


Using device: cpu


NameError: name 'MyModel' is not defined

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
#setting up path
base_path = "/content/drive/MyDrive/odss"
natural_path = os.path.join(base_path, "natural")
vits_path = os.path.join(base_path, "vits")

Glob - finds matching patterns , here trying to find all .wav files. . very *.wav wildcard coded.
coolect_wav_files will take 2 inputs , the directory (in this case natural and vits) and then label it 1 or 0





In [ ]:
from glob import glob

def collect_wav_files(root_dir, label):
    data = []
    for dataset in os.listdir(root_dir):
        dataset_path = os.path.join(root_dir, dataset)
        if not os.path.isdir(dataset_path):
            continue
        for speaker in os.listdir(dataset_path):
            speaker_path = os.path.join(dataset_path, speaker)
            wavs = glob(os.path.join(speaker_path, "*.wav"))
            for wav_path in wavs:
                data.append({
                    "speaker": speaker,
                    "dataset": dataset,
                    "path": wav_path,
                    "is_fake": label
                })
    return data

natural_data = collect_wav_files(natural_path, 0)
vits_data = collect_wav_files(vits_path, 1)


In [ ]:
import pandas as pd

speaker_info_path = "/content/drive/MyDrive/odss/speaker_info.csv"  # Update if needed
speaker_info_df = pd.read_csv(speaker_info_path)

In [ ]:

audio_df = pd.DataFrame(natural_data + vits_data)

In [ ]:

merged_df = audio_df.merge(speaker_info_df, on=["speaker", "dataset"], how="left")

In [ ]:
merged_df.tail()


,speaker,dataset,path,is_fake,gender,language
22059,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de
22060,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de
22061,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de
22062,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de
22063,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de


In [ ]:
!pip install praat-parselmouth

In [ ]:
import librosa
import numpy as np
import parselmouth

def extract_features(path):
    features = {}

    y, sr = librosa.load(path, sr=None)

    # MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    for i in range(13):
        features[f'mfcc_{i}_mean'] = np.mean(mfcc[i])
        features[f'mfcc_{i}_std'] = np.std(mfcc[i])

    # Pitch
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr)
    pitch_values = pitches[magnitudes > np.median(magnitudes)]
    features['pitch_mean'] = np.mean(pitch_values) if pitch_values.size else 0

    # Speech rate (ZCR)
    zcr = librosa.feature.zero_crossing_rate(y)
    features['speech_rate'] = np.mean(zcr)

    # Parselmouth (jitter, shimmer, HNR, formants)
    snd = parselmouth.Sound(path)

    try:
        pp = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 500)
        features['jitter'] = parselmouth.praat.call([snd, pp], "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
        features['shimmer'] = parselmouth.praat.call([snd, pp], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    except:
        features['jitter'] = np.nan
        features['shimmer'] = np.nan

    try:
        hnr_obj = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
        features['hnr'] = parselmouth.praat.call(hnr_obj, "Get mean", 0, 0)
    except:
        features['hnr'] = np.nan

    try:
        formant = parselmouth.praat.call(snd, "To Formant (burg)", 0.02, 5, 5500, 0.025, 50)
        f1_list, f2_list = [], []
        for t in np.arange(0.05, snd.duration, 0.01):  # Avoid time 0
            f1 = parselmouth.praat.call(formant, "Get value at time", 1, t, 'Hertz', 'Linear')
            f2 = parselmouth.praat.call(formant, "Get value at time", 2, t, 'Hertz', 'Linear')
            if f1: f1_list.append(f1)
            if f2: f2_list.append(f2)
        features['F1'] = np.mean(f1_list) if f1_list else np.nan
        features['F2'] = np.mean(f2_list) if f2_list else np.nan
    except:
        features['F1'] = np.nan
        features['F2'] = np.nan

    return features


In [ ]:
# deepcopy for log reg version - using the main df and adding required features for each model separately
log_reg_df = merged_df.copy()

# For new features that will be extracted
feature_columns = []

from tqdm import tqdm

for idx, row in tqdm(log_reg_df.iterrows(), total=len(log_reg_df)):

    try:
        feats = extract_features(row["path"])  # Correct function
        for k, v in feats.items():
            log_reg_df.at[idx, k] = v
            if k not in feature_columns:
                feature_columns.append(k)
    except Exception as e:
        print(f"Error in {row['path']}: {e}")

100%|██████████| 22064/22064 [3:28:26<00:00,  1.76it/s]


In [ ]:
log_reg_df.sample(5)

,speaker,dataset,path,is_fake,gender,language,mfcc_0_mean,mfcc_0_std,mfcc_1_mean,mfcc_1_std,...,mfcc_11_std,mfcc_12_mean,mfcc_12_std,pitch_mean,speech_rate,jitter,shimmer,hnr,F1,F2
18544,p317,vctk,/content/drive/MyDrive/odss/vits/vctk/p317/p31...,1,f,en,-452.209656,164.743179,68.878540,50.012478,...,7.482750,5.475425,6.985565,1954.032349,0.075643,NaN,NaN,12.376199,NaN,NaN
8684,p376,vctk,/content/drive/MyDrive/odss/natural/vctk/p376/...,0,m,en,-427.454468,177.984116,105.119484,38.374992,...,9.045292,0.170788,11.626082,851.715576,0.038292,NaN,NaN,10.004600,NaN,NaN
17888,p263,vctk,/content/drive/MyDrive/odss/vits/vctk/p263/p26...,1,m,en,-440.722504,179.006790,83.991989,69.180504,...,7.661402,5.241793,6.759166,1203.329712,0.042244,NaN,NaN,13.046357,NaN,NaN
15259,6670,hifi-tts,/content/drive/MyDrive/odss/vits/hifi-tts/6670...,1,m,en,-307.678833,141.038925,83.956528,53.497101,...,8.273189,-12.496811,10.712881,1255.044678,0.083181,NaN,NaN,13.920808,NaN,NaN
5725,p304,vctk,/content/drive/MyDrive/odss/natural/vctk/p304/...,0,m,en,-404.294952,163.535797,78.651741,59.326679,...,7.929339,7.800468,6.806385,1535.142700,0.080843,NaN,NaN,8.220338,NaN,NaN


In [ ]:
log_reg_df.to_pickle("log_reg_features.pkl")  # more efficient


In [ ]:
log_reg_df.to_csv("log_reg_features.csv", index=False)


In [ ]:
#PATCHING FROM HERE AS MISSING FEATURES AS NAN VALUES

In [ ]:
import parselmouth
import numpy as np

def extract_voice_quality_features(path):
    features = {}

    try:
        snd = parselmouth.Sound(path)

        try:
            pp = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 500)
            features['jitter'] = parselmouth.praat.call([snd, pp], "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
            features['shimmer'] = parselmouth.praat.call([snd, pp], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
        except:
            features['jitter'] = np.nan
            features['shimmer'] = np.nan

        try:
            formant = parselmouth.praat.call(snd, "To Formant (burg)", 0.02, 5, 5500, 0.025, 50)
            f1_list, f2_list = [], []
            for t in np.arange(0.05, snd.duration, 0.01):
                f1 = parselmouth.praat.call(formant, "Get value at time", 1, t, 'Hertz', 'Linear')
                f2 = parselmouth.praat.call(formant, "Get value at time", 2, t, 'Hertz', 'Linear')
                if f1: f1_list.append(f1)
                if f2: f2_list.append(f2)
            features['F1'] = np.mean(f1_list) if f1_list else np.nan
            features['F2'] = np.mean(f2_list) if f2_list else np.nan
        except:
            features['F1'] = np.nan
            features['F2'] = np.nan

    except Exception as e:
        print(f"Error in {path}: {e}")
        features = {'jitter': np.nan, 'shimmer': np.nan, 'F1': np.nan, 'F2': np.nan}

    return features


In [ ]:
from tqdm import tqdm

for idx, row in tqdm(log_reg_df.iterrows(), total=len(log_reg_df)):
    if any(pd.isna(row[col]) for col in ['jitter', 'shimmer', 'F1', 'F2']):
        feats = extract_voice_quality_features(row['path'])
        for k, v in feats.items():
            log_reg_df.at[idx, k] = v


100%|██████████| 22064/22064 [1:06:23<00:00,  5.54it/s]


In [ ]:
log_reg_df.sample(5)


NameError: name 'log_reg_df' is not defined

In [ ]:

import pandas as pd

log_reg_df = pd.read_csv("/content/drive/MyDrive/odss/log_reg_features.csv")



In [ ]:
!pip install opensmile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.0/996.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 21.6 MB/s eta 0:00:00


In [ ]:
from opensmile import Smile, FeatureSet, FeatureLevel
import numpy as np
from tqdm import tqdm


In [ ]:


# Initialize openSMILE
smile = Smile(
    feature_set=FeatureSet.ComParE_2016,
    feature_level=FeatureLevel.Functionals
)

# Patch missing jitter and shimmer values using openSMILE
def patch_with_opensmile(path):
    try:
        result = smile.process_file(path)
        row = result.iloc[0]
        jitter = row.get('jitterLocal_sma_linregc2', np.nan)
        shimmer = row.get('shimmerLocal_sma_linregc2', np.nan)
        return jitter, shimmer
    except Exception as e:
        print(f"[x] Failed for {path}: {e}")
        return np.nan, np.nan


In [ ]:
for idx, row in tqdm(log_reg_df.iterrows(), total=len(log_reg_df)):
    if pd.isna(row['jitter']) or pd.isna(row['shimmer']):
        jitter, shimmer = patch_with_opensmile(row['path'])
        log_reg_df.at[idx, 'jitter'] = jitter
        log_reg_df.at[idx, 'shimmer'] = shimmer


100%|██████████| 22064/22064 [5:13:10<00:00,  1.17it/s]


In [ ]:
log_reg_df.tail()

,speaker,dataset,path,is_fake,gender,language,mfcc_0_mean,mfcc_0_std,mfcc_1_mean,mfcc_1_std,...,mfcc_11_std,mfcc_12_mean,mfcc_12_std,pitch_mean,speech_rate,jitter,shimmer,hnr,F1,F2
22059,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-310.93870,120.694990,61.073853,64.459770,...,7.440020,3.355297,6.266328,1775.2618,0.176069,0.073048,0.139288,13.438156,NaN,NaN
22060,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-305.67650,124.017780,66.071620,73.535990,...,7.932513,5.520779,8.687095,1471.0764,0.170039,0.098338,0.223008,12.527261,NaN,NaN
22061,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-294.58383,111.261635,59.541496,57.305347,...,9.059581,4.042005,8.383234,1541.3538,0.169054,0.201576,0.189968,15.453236,NaN,NaN
22062,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-281.68268,140.744750,78.601280,71.481090,...,8.549084,3.425850,7.841340,1611.5330,0.151762,0.084946,0.181492,11.022446,NaN,NaN
22063,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-279.34973,121.508830,69.675600,73.611080,...,8.793427,2.227032,7.677840,1487.1865,0.177178,0.065603,0.154541,12.820001,NaN,NaN


In [ ]:
log_reg_df.to_csv("/content/log_reg_features_patched.csv", index=False)


In [ ]:
from google.colab import files
files.download("/content/log_reg_features_patched.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install praat-parselmouth


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 70.7 MB/s eta 0:00:00


In [ ]:
import librosa
import numpy as np
import scipy.signal

def get_formants_lpc(y, sr):
    try:
        y = y[:int(sr * 0.03)]  # just a slice (30ms)
        n_coeffs = 2 + sr // 1000
        a = scipy.signal.lpc(y, n_coeffs)
        roots = np.roots(a)
        roots = [r for r in roots if np.imag(r) >= 0.01]
        angz = np.arctan2(np.imag(roots), np.real(roots))
        formants = sorted(angz * (sr / (2 * np.pi)))
        return formants[:2] if len(formants) >= 2 else (np.nan, np.nan)
    except Exception as e:
        print("LPC fail:", e)
        return np.nan, np.nan


In [ ]:
y, sr = librosa.load("/content/drive/MyDrive/odss/natural/vctk/p234/p234_054.wav", sr=None)
f1, f2 = get_formants_lpc(y, sr)
print("F1:", f1, "F2:", f2)


LPC fail: module 'scipy.signal' has no attribute 'lpc'
F1: nan F2: nan


In [ ]:
##other approach

import librosa
import numpy as np

def lpc(signal, order):
    """Levinson-Durbin recursion to solve LPC"""
    from scipy.linalg import solve_toeplitz
    autocorr = np.correlate(signal, signal, mode='full')
    autocorr = autocorr[len(autocorr)//2:]
    R = autocorr[:order+1]
    r = R[1:]
    a = solve_toeplitz((R[:-1], R[:-1]), r)
    a = np.concatenate(([1], -a))
    return a


In [ ]:
import scipy.signal

def get_formants_lpc_amateur(y, sr):
    try:
        # Use first 30ms of audio
        y = y[:int(sr * 0.03)]
        y = y * np.hamming(len(y))  # window
        y = y - np.mean(y)

        # LPC order depends on sampling rate
        order = int(2 + sr / 1000)
        a = lpc(y, order)

        # Find roots
        roots = np.roots(a)
        roots = [r for r in roots if np.imag(r) >= 0.01]

        angz = np.arctan2(np.imag(roots), np.real(roots))
        formants = sorted(angz * (sr / (2 * np.pi)))
        return formants[:2] if len(formants) >= 2 else (np.nan, np.nan)
    except Exception as e:
        print("Formant LPC fail:", e)
        return np.nan, np.nan


In [ ]:
y, sr = librosa.load("/content/drive/MyDrive/odss/natural/vctk/p234/p234_054.wav", sr=None)
f1, f2 = get_formants_lpc_amateur(y, sr)
print("F1:", f1)
print("F2:", f2)


🎯 F1: 1184.931422536204
🎯 F2: 1838.4191475662699


In [ ]:
import pandas as pd

log_reg_df = pd.read_csv("/content/drive/MyDrive/odss/log_reg_features.csv")
log_reg_df

,speaker,dataset,path,is_fake,gender,language,mfcc_0_mean,mfcc_0_std,mfcc_1_mean,mfcc_1_std,...,mfcc_11_std,mfcc_12_mean,mfcc_12_std,pitch_mean,speech_rate,jitter,shimmer,hnr,F1,F2
0,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-272.38794,115.978830,97.005030,76.182000,...,12.981519,-12.861583,11.595129,1466.7181,0.130257,NaN,NaN,9.832871,NaN,NaN
1,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-299.20206,148.597900,67.028950,56.052290,...,8.924316,-9.231380,12.028031,1452.8362,0.116930,NaN,NaN,12.636739,NaN,NaN
2,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-271.15112,115.128390,109.935100,61.197582,...,10.310383,-8.451279,12.086157,1236.2042,0.117368,NaN,NaN,10.681270,NaN,NaN
3,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-325.16110,137.085100,71.334390,66.556015,...,11.130038,-8.204823,12.830888,1823.1019,0.125980,NaN,NaN,12.730017,NaN,NaN
4,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-250.64236,104.512190,67.433770,54.104633,...,9.660391,-10.229192,11.922198,1689.7440,0.154225,NaN,NaN,11.085720,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22059,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-310.93870,120.694990,61.073853,64.459770,...,7.440020,3.355297,6.266328,1775.2618,0.176069,NaN,NaN,13.438156,NaN,NaN
22060,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-305.67650,124.017780,66.071620,73.535990,...,7.932513,5.520779,8.687095,1471.0764,0.170039,NaN,NaN,12.527261,NaN,NaN
22061,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-294.58383,111.261635,59.541496,57.305347,...,9.059581,4.042005,8.383234,1541.3538,0.169054,NaN,NaN,15.453236,NaN,NaN
22062,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-281.68268,140.744750,78.601280,71.481090,...,8.549084,3.425850,7.841340,1611.5330,0.151762,NaN,NaN,11.022446,NaN,NaN


In [ ]:
import librosa
import numpy as np

def lpc(signal, order):
    from scipy.linalg import solve_toeplitz
    autocorr = np.correlate(signal, signal, mode='full')
    autocorr = autocorr[len(autocorr)//2:]
    R = autocorr[:order+1]
    r = R[1:]
    a = solve_toeplitz((R[:-1], R[:-1]), r)
    a = np.concatenate(([1], -a))
    return a

def get_formants_lpc_amateur(y, sr):
    try:
        y = y[:int(sr * 0.05)]
        y = y * np.hamming(len(y))
        y = y - np.mean(y)

        # Skip if signal is too flat
        if np.std(y) < 1e-5:
            return np.nan, np.nan

        order = int(2 + sr / 1000)
        a = lpc(y, order)

        roots = np.roots(a)
        roots = [r for r in roots if np.imag(r) >= 0.01]
        angz = np.arctan2(np.imag(roots), np.real(roots))
        formants = sorted(angz * (sr / (2 * np.pi)))
        return formants[:2] if len(formants) >= 2 else (np.nan, np.nan)

    except Exception as e:
        print("fail on formants:", e)
        return np.nan, np.nan



In [ ]:
from tqdm import tqdm
import os

In [ ]:
for index, row in tqdm(log_reg_df.iterrows(), total=len(log_reg_df)):
    if pd.isna(row['F1']) or pd.isna(row['F2']):
        file_path = row['path']
        if os.path.exists(file_path):
            try:
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                f1, f2 = get_formants_lpc_amateur(audio_data, sample_rate)
                log_reg_df.at[index, 'F1'] = f1
                log_reg_df.at[index, 'F2'] = f2
            except Exception as error:
                print("Error processing file:", file_path)
                print("Reason:", error)
        else:
            print("File not found:", file_path)


100%|██████████| 22064/22064 [1:20:49<00:00,  4.55it/s]


In [ ]:
log_reg_df

,speaker,dataset,path,is_fake,gender,language,mfcc_0_mean,mfcc_0_std,mfcc_1_mean,mfcc_1_std,...,mfcc_11_std,mfcc_12_mean,mfcc_12_std,pitch_mean,speech_rate,jitter,shimmer,hnr,F1,F2
0,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-272.38794,115.978830,97.005030,76.182000,...,12.981519,-12.861583,11.595129,1466.7181,0.130257,NaN,NaN,9.832871,482.867611,1641.274239
1,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-299.20206,148.597900,67.028950,56.052290,...,8.924316,-9.231380,12.028031,1452.8362,0.116930,NaN,NaN,12.636739,793.978415,1594.566758
2,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-271.15112,115.128390,109.935100,61.197582,...,10.310383,-8.451279,12.086157,1236.2042,0.117368,NaN,NaN,10.681270,1608.138021,1671.665541
3,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-325.16110,137.085100,71.334390,66.556015,...,11.130038,-8.204823,12.830888,1823.1019,0.125980,NaN,NaN,12.730017,647.187769,1268.787594
4,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-250.64236,104.512190,67.433770,54.104633,...,9.660391,-10.229192,11.922198,1689.7440,0.154225,NaN,NaN,11.085720,550.164814,1259.709604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22059,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-310.93870,120.694990,61.073853,64.459770,...,7.440020,3.355297,6.266328,1775.2618,0.176069,NaN,NaN,13.438156,NaN,NaN
22060,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-305.67650,124.017780,66.071620,73.535990,...,7.932513,5.520779,8.687095,1471.0764,0.170039,NaN,NaN,12.527261,NaN,NaN
22061,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-294.58383,111.261635,59.541496,57.305347,...,9.059581,4.042005,8.383234,1541.3538,0.169054,NaN,NaN,15.453236,NaN,NaN
22062,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-281.68268,140.744750,78.601280,71.481090,...,8.549084,3.425850,7.841340,1611.5330,0.151762,NaN,NaN,11.022446,NaN,NaN


In [ ]:
log_reg_df.to_csv("/content/log_reg_features_formants.csv", index=False)


from google.colab import files
files.download("/content/log_reg_features_formants.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

import pandas as pd

# Load the one with jitter, shimmer, HNR
df_jitter_shimmer = pd.read_csv("/content/drive/MyDrive/odss/log_reg_features_patched.csv")



In [34]:
from google.colab import drive

drive.flush_and_unmount()


In [35]:
drive.mount('/content/drive')


Mounted at /content/drive


In [36]:
import os
os.listdir("/content/drive/MyDrive/odss")


['LICENSE',
 'speaker_info analysis.csv',
 'speaker_info.csv',
 'transcripts.psv',
 'Tab1.twb',
 'fastpitch-hifigan',
 'vits',
 'natural',
 'log_reg_features.csv',
 'log_reg_features_patched.csv',
 'log_reg_features_formants.csv']

In [37]:
import pandas as pd

# Load both CSVs from Drive
df_jitter_shimmer = pd.read_csv("/content/drive/MyDrive/odss/log_reg_features_patched.csv")
df_formants = pd.read_csv("/content/drive/MyDrive/odss/log_reg_features_formants.csv")

# Merge on 'path'
df_complete = pd.merge(
    df_jitter_shimmer,
    df_formants[['path', 'F1', 'F2']],
    on='path',
    how='left'
)

# Save final version to Drive
df_complete.to_csv("/content/drive/MyDrive/odss/log_reg_features_complete.csv", index=False)


In [41]:
df_complete


,speaker,dataset,path,is_fake,gender,language,mfcc_0_mean,mfcc_0_std,mfcc_1_mean,mfcc_1_std,...,mfcc_12_std,pitch_mean,speech_rate,jitter,shimmer,hnr,F1_x,F2_x,F1_y,F2_y
0,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-272.38794,115.978830,97.005030,76.182000,...,11.595129,1466.7181,0.130257,0.064248,0.172149,9.832871,NaN,NaN,559.385154,1649.042463
1,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-299.20206,148.597900,67.028950,56.052290,...,12.028031,1452.8362,0.116930,0.006966,0.164342,12.636739,NaN,NaN,794.319768,1584.265895
2,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-271.15112,115.128390,109.935100,61.197582,...,12.086157,1236.2042,0.117368,0.152547,0.280916,10.681270,NaN,NaN,499.772616,1669.927625
3,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-325.16110,137.085100,71.334390,66.556015,...,12.830888,1823.1019,0.125980,0.086650,0.178736,12.730017,NaN,NaN,1207.080864,2238.898504
4,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-250.64236,104.512190,67.433770,54.104633,...,11.922198,1689.7440,0.154225,0.185120,0.296203,11.085720,NaN,NaN,592.331004,1118.893148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22059,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-310.93870,120.694990,61.073853,64.459770,...,6.266328,1775.2618,0.176069,0.073048,0.139288,13.438156,NaN,NaN,722.592263,1631.991611
22060,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-305.67650,124.017780,66.071620,73.535990,...,8.687095,1471.0764,0.170039,0.098338,0.223008,12.527261,NaN,NaN,156.706988,853.565574
22061,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-294.58383,111.261635,59.541496,57.305347,...,8.383234,1541.3538,0.169054,0.201576,0.189968,15.453236,NaN,NaN,793.821791,2039.542725
22062,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-281.68268,140.744750,78.601280,71.481090,...,7.841340,1611.5330,0.151762,0.084946,0.181492,11.022446,NaN,NaN,690.813451,1520.805832


In [43]:
# Drop old formant columns - EMPTY
df_complete = df_complete.drop(columns=['F1_x', 'F2_x'])

# Rename the good ones
df_complete = df_complete.rename(columns={'F1_y': 'F1', 'F2_y': 'F2'})


KeyError: "['F1_x', 'F2_x'] not found in axis"

In [55]:
df_complete

,speaker,dataset,path,is_fake,gender,language,mfcc_0_mean,mfcc_0_std,mfcc_1_mean,mfcc_1_std,...,mfcc_11_std,mfcc_12_mean,mfcc_12_std,pitch_mean,speech_rate,jitter,shimmer,hnr,F1,F2
0,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-272.38794,115.978830,97.005030,76.182000,...,12.981519,-12.861583,11.595129,1466.7181,0.130257,0.064248,0.172149,9.832871,559.385154,1649.042463
1,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-299.20206,148.597900,67.028950,56.052290,...,8.924316,-9.231380,12.028031,1452.8362,0.116930,0.006966,0.164342,12.636739,794.319768,1584.265895
2,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-271.15112,115.128390,109.935100,61.197582,...,10.310383,-8.451279,12.086157,1236.2042,0.117368,0.152547,0.280916,10.681270,499.772616,1669.927625
3,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-325.16110,137.085100,71.334390,66.556015,...,11.130038,-8.204823,12.830888,1823.1019,0.125980,0.086650,0.178736,12.730017,1207.080864,2238.898504
4,Bernd_Ungerer,hui-acg,/content/drive/MyDrive/odss/natural/hui-acg/Be...,0,m,de,-250.64236,104.512190,67.433770,54.104633,...,9.660391,-10.229192,11.922198,1689.7440,0.154225,0.185120,0.296203,11.085720,592.331004,1118.893148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22059,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-310.93870,120.694990,61.073853,64.459770,...,7.440020,3.355297,6.266328,1775.2618,0.176069,0.073048,0.139288,13.438156,722.592263,1631.991611
22060,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-305.67650,124.017780,66.071620,73.535990,...,7.932513,5.520779,8.687095,1471.0764,0.170039,0.098338,0.223008,12.527261,156.706988,853.565574
22061,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-294.58383,111.261635,59.541496,57.305347,...,9.059581,4.042005,8.383234,1541.3538,0.169054,0.201576,0.189968,15.453236,793.821791,2039.542725
22062,Hokuspokus,hui-acg,/content/drive/MyDrive/odss/vits/hui-acg/Hokus...,1,f,de,-281.68268,140.744750,78.601280,71.481090,...,8.549084,3.425850,7.841340,1611.5330,0.151762,0.084946,0.181492,11.022446,690.813451,1520.805832


In [56]:
selected_features = [
    'pitch_mean', 'speech_rate',
    'mfcc_0_mean', 'mfcc_1_mean', 'mfcc_2_mean', 'mfcc_3_mean', 'mfcc_4_mean',
    'mfcc_5_mean', 'mfcc_6_mean', 'mfcc_7_mean', 'mfcc_8_mean', 'mfcc_9_mean',
    'jitter', 'shimmer', 'hnr', 'F1', 'F2'
]



In [58]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Drop rows with missing features and reset index
df_logreg = df_complete.dropna(subset=selected_features).reset_index(drop=True)

# Feature matrix and target
X = df_logreg[selected_features]
y = df_logreg['is_fake']

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# split, just X and y
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(
    X_scaled,
    y,
    df_logreg.index,
    test_size=0.2,
    stratify=y,
    random_state=42
)




In [59]:
meta_test = df_logreg.loc[test_idx, ['speaker', 'dataset', 'path', 'gender', 'language']]


In [60]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [62]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 1, 0])

In [63]:
from sklearn.metrics import classification_report

# Show basic metrics
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.65      0.65      0.65      2172
           1       0.65      0.65      0.65      2155

    accuracy                           0.65      4327
   macro avg       0.65      0.65      0.65      4327
weighted avg       0.65      0.65      0.65      4327



In [64]:
results_df = meta_test.copy()
results_df['y_true'] = y_test.values
results_df['y_pred'] = y_pred


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pandas as pd

# Define selected features for logistic regression
selected_features = [
    'pitch_mean', 'speech_rate',
    'mfcc_0_mean', 'mfcc_1_mean', 'mfcc_2_mean', 'mfcc_3_mean', 'mfcc_4_mean',
    'mfcc_5_mean', 'mfcc_6_mean', 'mfcc_7_mean', 'mfcc_8_mean', 'mfcc_9_mean',
    'jitter', 'shimmer', 'hnr', 'F1', 'F2'
]

# Drop rows with missing values in selected features
df_logreg = df_complete.dropna(subset=selected_features).reset_index(drop=True)

# Extract X (features), y (target), and metadata
X = df_logreg[selected_features]
y = df_logreg['is_fake']
metadata = df_logreg[['speaker', 'dataset', 'path', 'gender', 'language']]

# Normalize X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split with stratification
X_train, X_test, y_train, y_test, meta_train, meta_test = train_test_split(
    X_scaled, y, metadata,
    test_size=0.2, stratify=y, random_state=42
)

# Train logistic regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Create results with true and predicted labels
results_df = meta_test.copy()
results_df['y_true'] = y_test.values
results_df['y_pred'] = y_pred

# Add all feature columns for test set (excluding is_fake)
X_test_df = pd.DataFrame(X_test, columns=selected_features)
results_full = pd.concat([results_df.reset_index(drop=True), X_test_df], axis=1)

# Save to CSV
results_full.to_csv("/content/logreg_test_results.csv", index=False)


In [67]:
from google.colab import files
files.download("/content/logreg_test_results.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:
summary_by_gender = results_full.groupby('gender').apply(
    lambda g: pd.Series({
        'total': len(g),
        'true_real': ((g['y_true'] == 0) & (g['y_pred'] == 0)).sum(),
        'true_fake': ((g['y_true'] == 1) & (g['y_pred'] == 1)).sum(),
        'false_real': ((g['y_true'] == 1) & (g['y_pred'] == 0)).sum(),
        'false_fake': ((g['y_true'] == 0) & (g['y_pred'] == 1)).sum(),
        'accuracy': (g['y_true'] == g['y_pred']).mean()
    })
).reset_index()

print(summary_by_gender)


  gender   total  true_real  true_fake  false_real  false_fake  accuracy
0      f  2082.0      554.0      826.0       204.0       498.0  0.662824
1      m  2245.0      868.0      568.0       557.0       252.0  0.639644


<ipython-input-68-0876bbb9bacb>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary_by_gender = results_full.groupby('gender').apply(


In [69]:
female = summary_by_gender[summary_by_gender['gender'] == 'f'].iloc[0]
male = summary_by_gender[summary_by_gender['gender'] == 'm'].iloc[0]


likewise i plan to use all metrics here to see their difference.  lower error and higher accuracy . then just conclude which is more biased. very obvious its females here. :)

Accuracy difference between genders: 2.32%
